In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

In [9]:
appName = "nlp_darija"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/11/20 22:52:59 WARN Utils: Your hostname, adham-inspiron5502 resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
22/11/20 22:52:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/20 22:53:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
darija_csv = spark.read.csv('Train.csv', inferSchema=True, header=True)
darija_csv.show(truncate=False, n=10)

+---+---------------------------------------------------------------------------+---------+
|ID |Feed                                                                       |Sentiment|
+---+---------------------------------------------------------------------------+---------+
|1  |m3alem bourjilia w illi ma yefehmouch yelzmou ya9ra 5 ans plus le bac      |Positive |
|2  |Ya m3alllam                                                                |Positive |
|3  |Ma7lek! Zin ou fannena 7loua.                                              |Positive |
|4  |hhhhh ya bliiiiiiiida ya Hanen                                             |Negative |
|5  |Nikraha w ma5yebha pffff                                                   |Negative |
|6  |3sal denya :*                                                              |Positive |
|7  |سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شرهة في شباب باش يزيد يقرأ أكثر|Positive |
|8  |Ya mama mali masta lasta :p :p                                             

In [33]:
darija_csv=darija_csv.replace(to_replace="Positive",value="1")
darija_csv=darija_csv.replace(to_replace="Negative",value="0")

In [36]:
darija_csv.show(truncate=False, n=10)

+---+---------------------------------------------------------------------------+---------+
|ID |Feed                                                                       |Sentiment|
+---+---------------------------------------------------------------------------+---------+
|1  |m3alem bourjilia w illi ma yefehmouch yelzmou ya9ra 5 ans plus le bac      |1        |
|2  |Ya m3alllam                                                                |1        |
|3  |Ma7lek! Zin ou fannena 7loua.                                              |1        |
|4  |hhhhh ya bliiiiiiiida ya Hanen                                             |0        |
|5  |Nikraha w ma5yebha pffff                                                   |0        |
|6  |3sal denya :*                                                              |1        |
|7  |سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شرهة في شباب باش يزيد يقرأ أكثر|1        |
|8  |Ya mama mali masta lasta :p :p                                             

In [57]:
data = darija_csv.select("Feed", col("Sentiment").cast("Int").alias("label"))
data.show(truncate = False,n=5)

+---------------------------------------------------------------------+-----+
|Feed                                                                 |label|
+---------------------------------------------------------------------+-----+
|m3alem bourjilia w illi ma yefehmouch yelzmou ya9ra 5 ans plus le bac|1    |
|Ya m3alllam                                                          |1    |
|Ma7lek! Zin ou fannena 7loua.                                        |1    |
|hhhhh ya bliiiiiiiida ya Hanen                                       |0    |
|Nikraha w ma5yebha pffff                                             |0    |
+---------------------------------------------------------------------+-----+
only showing top 5 rows



In [59]:
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 8316 ; Testing data rows: 3563


In [62]:
tokenizer = Tokenizer(inputCol="Feed", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

22/11/20 23:25:24 ERROR Executor: Exception in task 0.0 in stage 26.0 (TID 22)
org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$3406/1419433049: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:364)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions

Py4JJavaError: An error occurred while calling o196.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 22) (192.168.122.1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$3406/1419433049: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.Spar

In [63]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

22/11/20 23:25:51 ERROR Executor: Exception in task 0.0 in stage 27.0 (TID 23)
org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$3406/1419433049: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:364)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions

Py4JJavaError: An error occurred while calling o220.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 23) (192.168.122.1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$3406/1419433049: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.Spar